In [6]:
import os
import pickle
import numpy as np
import pandas as pd
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.metrics.pairwise import cosine_similarity



In [1]:
import nltk
import numpy as np
#Given an object named 'texts' (a list of tokenized texts, ie a list of lists of tokens)
#from gensim.test.utils import common_texts as texts

# Create a corpus from a list of lists of tokens
from gensim.corpora.dictionary import Dictionary

from gensim.utils import simple_preprocess
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
nltk.download('stopwords')

texts=[]
file = open("TokenVieuxN.txt", "r")
lines = file.readlines()
file.close()

#print(stopwords.words())

for line in lines:
  line=line.strip()
  lt=line.split(",")
#Potential ill-character cleaning
  for i in range(len(lt)):
    lt[i]=lt[i].replace('[','')
    lt[i]=lt[i].replace(']','')
    lt[i]=lt[i].replace('"','')
    lt[i]=lt[i].replace('\n','')
    lt[i]=lt[i].replace(' ', '')
#End : Potential ill-characters cleaning
# print(lt)
  ltc=[word for word in lt if not word in stopwords.words()] # Remove Stopwords activated
#  print("C", ltc)
  texts.append(lt)

#for text in texts:
  #print(text)

#Here set the number of topics(to be changed if necessary)
nb=10
  
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]
#print(corpus)

# Print dictionnary # ***********
# for i in id2word:
#   print(i, id2word[i])

# Train the lda model on the corpus.
from gensim.models import LdaModel
lda = LdaModel(corpus, num_topics=nb)

# Print topic descrition
for i in range(0, nb-1):
  value=lda.get_topic_terms(i)
#  print(value)
  # print("Topic ", i) # *******************
  # for j in value:
  #   print(id2word[j[0]], " - P=", j[1])
  # print() # *******************

from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
# print('Coherence= ', coherence_lda) # *******************************

# from gensim.utils import simple_preprocess


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/krtucho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
doc_topics = lda.get_document_topics(corpus)
# doc_topic = []
# for topic in doc_topics:
#     for item in topic:
#         doc_topic.append(item)

# # doc_topics = [inside_doc_topic for inside_doc_topic in [doc_topic for doc_topic in doc_topics]]

doc_topics

In [4]:
# Get the topic distribution for each document

most_typical_docs = {}
for i, doc_topic in enumerate(doc_topics):
    for item in doc_topic: 
        topic, prob = item
        if most_typical_docs.__contains__(topic):
            if most_typical_docs[topic][1] < prob:
                most_typical_docs[topic] = (topic, prob, i)
        else:
            most_typical_docs[topic] = (topic, prob, i)
        
print("Typical Document for each topic, topic proportion (or probability) in documents") 
for i, prob, doc in most_typical_docs.values():
    print(f"Topic {i}: {doc} :  \nwith prob: {prob}")
    print()

Typical Document for each topic, topic proportion (or probability) in documents
Topic 5: 1 :  
with prob: 0.994229793548584

Topic 2: 3 :  
with prob: 0.9962176084518433

Topic 8: 5 :  
with prob: 0.9965642094612122

Topic 7: 6 :  
with prob: 0.9959448575973511

Topic 9: 8 :  
with prob: 0.9937487244606018



In [5]:
# Get the topic distribution for each document
doc_topics = lda.get_document_topics(corpus)

# print(doc_topics)
# doc_topics
# for doc_topic in doc_topics:
#     print(doc_topic)

# Get the most typical document for each topic based on cosine similarity
topic_words = [lda.get_topic_terms(i, topn=10) for i in range(nb)]
doc_vectors = np.zeros((len(texts), nb))
for i, doc in enumerate(texts):
    doc_term_vec = id2word.doc2bow(doc)
    for j, prob in lda[doc_term_vec]:
        doc_vectors[i][j] = prob
        # print("Prob: ", prob)



most_typical_docs = []
for topic in range(nb):
    topic_vec = np.zeros((1, nb))
    for word, _ in topic_words[topic]:
        if word in id2word.token2id:
            topic_vec[0][lda.get_term_topics(word)[0][0]] = 1
    similarity_scores = cosine_similarity(doc_vectors, topic_vec)
    # print("sim Scores: ", similarity_scores)
    most_typical_doc_index = np.argmax(similarity_scores)
    # print("topic: ", topic, "mtdi: ", most_typical_doc_index)
    most_typical_docs.append(texts[most_typical_doc_index])

# Print the most typical document for each topic
print("Typical Document for each topic, cosine_similarity") 
for i, doc in enumerate(most_typical_docs):
    print(f"Topic {i}: {doc}")
    print()

Typical Document for each topic, cosine_similarity
Topic 0: ['abstract', 'the', 'accumulation', 'of', 'nonenzymatic', 'modifications', 'on', 'both', 'dna', 'and', 'protein', 'molecules', 'under', 'the', 'attack', 'of', 'reactive', 'oxygen', 'species', 'ros', 'is', 'one', 'of', 'the', 'most', 'possible', 'factors', 'responsible', 'for', 'the', 'functional', 'deterioration', 'in', 'aged', 'cells', 'direct', 'protein', 'modifications', 'as', 'well', 'as', 'dna', 'damages', 'may', 'be', 'detectable', 'in', 'part', 'by', 'proteome', 'analysis', 'if', 'the', 'gene', 'expression', 'is', 'affected', 'by', 'the', 'damages', 'on', 'dna', 'the', 'novel', 'term', '“proteome”', 'which', 'is', 'a', 'compound', 'of', '“protein”', 'and', '“genome”', 'means', 'a', 'whole', 'set', 'of', 'proteins', 'expressed', 'in', 'a', 'tissue', 'or', 'a', 'cell', 'strain', 'to', 'be', 'investigated', 'proteomics', 'is', 'a', 'methodology', 'for', 'analyzing', 'proteomes', 'in', 'proteomics', 'twodimensional', 'gel',

In [6]:
for i in range(0, nb-1):
  value=lda.get_topic_terms(i)
  topic_words = [id2word[idx] for idx, _ in value]
  print("Topic ", i, ": ", topic_words)
  
  #Compute cosine similarity between topic words and each document
  for doc in texts:
    vec1 = id2word.doc2bow(doc)
    vec2 = id2word.doc2bow(topic_words)
    
    #Compute cosine similarity
    dot_product = 0
    doc_norm = 0
    topic_norm = 0
    # print(vec1)
    # print(vec2)
    for word_id, freq in vec1:
        # print(vec2[word_id])
        try:
            dot_product += freq * vec2[word_id]
            doc_norm += freq ** 2
        except:
           continue
    for word_id, freq in vec2:
        topic_norm += freq ** 2
    cosine_sim = dot_product / (doc_norm ** 0.5 * topic_norm ** 0.5) if  (doc_norm ** 0.5 * topic_norm ** 0.5) > 0 else 1
    
    print("Doc {} and Topic {} cosine similarity is {}".format(texts.index(doc), i, cosine_sim))

Topic  0 :  ['the', 'of', 'and', 'in', 'to', 'index', 'neurons', 'has', 'used', 'for']
Doc 0 and Topic 0 cosine similarity is 1
Doc 1 and Topic 0 cosine similarity is 1
Doc 2 and Topic 0 cosine similarity is 1
Doc 3 and Topic 0 cosine similarity is 1
Doc 4 and Topic 0 cosine similarity is 1
Doc 5 and Topic 0 cosine similarity is 1
Doc 6 and Topic 0 cosine similarity is 1
Doc 7 and Topic 0 cosine similarity is 1
Doc 8 and Topic 0 cosine similarity is 1
Doc 9 and Topic 0 cosine similarity is 1
Topic  1 :  ['and', 'of', 'the', 'to', 'for', 'in', 'knowledge', 'alzheimeraposs', 'methods', 'a']
Doc 0 and Topic 1 cosine similarity is 1
Doc 1 and Topic 1 cosine similarity is 1
Doc 2 and Topic 1 cosine similarity is 1
Doc 3 and Topic 1 cosine similarity is 1
Doc 4 and Topic 1 cosine similarity is 1
Doc 5 and Topic 1 cosine similarity is 1
Doc 6 and Topic 1 cosine similarity is 1
Doc 7 and Topic 1 cosine similarity is 1
Doc 8 and Topic 1 cosine similarity is 1
Doc 9 and Topic 1 cosine similarity

In [7]:
topic_counts = {}
sums = 0
for doc_topic in doc_topics:
    for item in doc_topic: 
        topic, prob = item
        if topic_counts.__contains__(str(topic)):
            topic_counts[str(topic)] = topic_counts[str(topic)]+1
        else:
            topic_counts[str(topic)] = 1
        sums += 1

rounds = [(topic, round(topic_count/sums, 4)) for (topic, topic_count) in topic_counts.items()]
print(rounds)

[('5', 0.4), ('2', 0.2), ('8', 0.2), ('7', 0.1), ('9', 0.1)]


TokenVieuxM.txt

As a result of taking into account the proportion of topics in documents, in one of the program's runs it was found that out of a total of 8 topics:

Typical Document for each topic, topic proportion (or probability) in documents 
Topic 0: Document 0 with prob: 0.9937055706977844

Topic 5: Document 1 with prob: 0.9942291975021362

Topic 1: Document 2 with prob: 0.9933327436447144

Topic 3: Document 3 with prob: 0.7552105784416199   

Topic 4: Document 3 with prob: 0.2414269596338272

Topic 8: Document 4 with prob: 0.9957541227340698

In the case of measuring cosine similarity, it was found that for each document the cosine similarity was 1 with all topics. Therefore, any document can be chosen to represent any of the topics.

TokenVieuxN.txt

As a result of taking into account the proportion of topics in documents, in one of the program's runs it was found that out of a total of 9 topics:

Topic 5: 1 :  
with prob: 0.994229793548584

Topic 2: 3 :  
with prob: 0.9962176084518433

Topic 8: 5 :  
with prob: 0.9965642094612122

Topic 7: 6 :  
with prob: 0.9959448575973511

Topic 9: 8 :  
with prob: 0.9937487244606018

In the case of measuring cosine similarity, it was found that for each document the cosine similarity was 1 with all topics. Therefore, any document can be chosen to represent any of the topics.